In [9]:
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [10]:
!ls data/pickles

hs_data_16-18.pickle


In [61]:
with open('data/pickles/hs_data_16-18.pickle', 'rb') as f:
    df = pickle.load(f)

In [62]:
df['School_Year']

0      School Year 2016-2017
1      School Year 2017-2018
2      School Year 2016-2017
3      School Year 2017-2018
4      School Year 2016-2017
5      School Year 2017-2018
6      School Year 2016-2017
7      School Year 2017-2018
8      School Year 2016-2017
9      School Year 2017-2018
10     School Year 2016-2017
11     School Year 2017-2018
               ...          
227    School Year 2016-2017
228    School Year 2017-2018
229    School Year 2016-2017
230    School Year 2017-2018
231    School Year 2016-2017
232    School Year 2017-2018
233    School Year 2016-2017
234    School Year 2017-2018
235    School Year 2016-2017
236    School Year 2017-2018
237    School Year 2016-2017
238    School Year 2017-2018
Name: School_Year, Length: 239, dtype: object

In [63]:
print(list(df))

['ADA_Accessible', 'Address', 'Administrator', 'Administrator_Title', 'After_School_Hours', 'Attendance_Boundaries', 'Average_ACT_School', 'Bilingual_Services', 'CPS_School_Profile', 'City', 'Classification_Description', 'Classroom_Languages', 'Closed_For_Enrollment_Date', 'College_Enrollment_Rate_Mean', 'College_Enrollment_Rate_School', 'Demographic_Description', 'Dress_Code', 'Earliest_Drop_Off_Time', 'Facebook', 'Fax', 'Fifth_Contact_Name', 'Fifth_Contact_Title', 'Finance_ID', 'Fourth_Contact_Name', 'Fourth_Contact_Title', 'Freshman_Start_End_Time', 'Grades_Offered', 'Grades_Offered_All', 'Graduation_Rate_Mean', 'Graduation_Rate_School', 'Hard_Of_Hearing', 'Is_Elementary_School', 'Is_GoCPS_Elementary', 'Is_GoCPS_High_School', 'Is_GoCPS_Participant', 'Is_GoCPS_PreK', 'Is_High_School', 'Is_Middle_School', 'Is_Pre_School', 'Kindergarten_School_Day', 'Legacy_Unit_ID', 'Location', 'Long_Name', 'Mean_ACT', 'Network', 'Open_For_Enrollment_Date', 'Overall_Rating', 'Phone', 'Pinterest', 'Pre

In [64]:
df.drop(columns = ['Address', 'Administrator_Title', 'CPS_School_Profile', 
                   'City', 'Classification_Description', 'Closed_For_Enrollment_Date',
                  'College_Enrollment_Rate_Mean', 'College_Enrollment_Rate_School',
                  'Demographic_Description', 'Earliest_Drop_Off_Time', 'Facebook', 
                  'Fax', 'Fifth_Contact_Name', 'Fifth_Contact_Title', 'Finance_ID', 
                  'Fourth_Contact_Name', 'Fourth_Contact_Title', 'Freshman_Start_End_Time',
                  'Grades_Offered', 'Graduation_Rate_Mean', 'Is_Elementary_School', 
                  'Is_GoCPS_Elementary', 'Is_GoCPS_High_School', 'Is_GoCPS_PreK', 
                  'Is_High_School', 'Is_Middle_School', 'Is_Pre_School', 'Kindergarten_School_Day',
                  'Legacy_Unit_ID', 'Long_Name', 'Mean_ACT', 'Open_For_Enrollment_Date', 
                  'Overall_Rating', 'Phone', 'Pinterest', 'PreK_School_Day', 'PreSchool_Inclusive', 
                  'Preschool_Instructional', 'Primary_Category', 'Rating_Statement', 'Rating_Status', 
                  'Refugee_Services', 'School_Type','Secondary_Contact', 'Secondary_Contact_Title', 
                  'Seventh_Contact_Name', 'Seventh_Contact_Title', 'Short_Name', 'Short_Name', 
                  'Significantly_Modified', 'Sixth_Contact_Name', 'Sixth_Contact_Title', 'State',
                  'Statistics_Description', 'Summary', 'Third_Contact_Name', 'Third_Contact_Title', 
                  'Title_1_Eligible', 'Transportation_El', 'Twitter', 'Visual_Impairments', 'Website',
                  'Youtube', 'Zip_x'], inplace = True)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 0 to 238
Data columns (total 36 columns):
ADA_Accessible                             121 non-null object
Administrator                              239 non-null object
After_School_Hours                         143 non-null object
Attendance_Boundaries                      239 non-null object
Average_ACT_School                         117 non-null float64
Bilingual_Services                         228 non-null object
Classroom_Languages                        228 non-null object
Dress_Code                                 239 non-null object
Grades_Offered_All                         239 non-null object
Graduation_Rate_School                     239 non-null float64
Hard_Of_Hearing                            11 non-null object
Is_GoCPS_Participant                       118 non-null object
Location                                   239 non-null object
Network                                    116 non-null object
School_Hours

In [71]:
df_2016 = df[df['School_Year'] == 'School Year 2016-2017']
df_2016_income = pd.read_csv('data/meanHHincome_2016.csv', header=None)
df_2016_income.columns = ['Zip_y', 'Zip_Mean_Income']
df_2016 = pd.merge(df_2016, df_2016_income, on = 'Zip_y' )

In [72]:
df_2016

,ADA_Accessible,Administrator,After_School_Hours,Attendance_Boundaries,Average_ACT_School,Bilingual_Services,Classroom_Languages,Dress_Code,Grades_Offered_All,Graduation_Rate_School,Hard_Of_Hearing,Is_GoCPS_Participant,Location,Network,School_Hours,School_ID,School_Latitude,School_Longitude,School_Year,Student_Count_Asian,Student_Count_Asian_Pacific_Islander,Student_Count_Black,Student_Count_English_Learners,Student_Count_Ethnicity_Not_Available,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Hispanic,Student_Count_Low_Income,Student_Count_Multi,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Special_Ed,Student_Count_Total,Student_Count_White,Transportation_Bus,Transportation_Metra,Zip_y,Zip_Mean_Income
0,Generally accessible,Juan Carlos Ocon,2:40 to 8:00 P.M.,Y,16.5,Y,"French, Spanish",N,"9,10,11,12",80.9,NaN,NaN,"1450 W CERMAK RD\nChicago, Illinois 60608\n(41...",NaN,7:45 AM - 2:35 PM,609764,41.853,-87.664,School Year 2016-2017,5,0,58,331,11,0,1644,1659,3,6,0,281,1739,12,"9, 21, 60",NaN,60608.0,"53,353"
1,No/unknown accessibility,Dr. Hillyn Senuholtz,3:45 PM - 5:30 PM,N,16.7,Y,Spanish,Y,"9,10,11,12",82.8,NaN,NaN,"2520 S WESTERN AVE\nChicago, Illinois 60608\n(...",NaN,7:45AM - 3:44PM,400104,41.846,-87.687,School Year 2016-2017,0,0,24,169,0,0,714,720,0,1,0,146,742,3,"49, 60",NaN,60608.0,"53,353"
2,No/unknown accessibility,Sharnette Sims,NaN,N,NaN,NaN,NaN,N,"9,10,11,12",1.7,NaN,NaN,"2700 S CALIFORNIA AVE\nChicago, Illinois 60608...",NaN,7:00 am - 4:00 pm,609748,41.843,-87.695,School Year 2016-2017,0,0,154,2,0,0,36,174,0,3,0,58,199,6,NaN,NaN,60608.0,"53,353"
3,Generally accessible,Tressie McDonough,NaN,N,22.2,Y,"Spanish, Spanish for Heritage Speakers",Y,"9,10,11,12",83.3,NaN,NaN,"1231 S DAMEN AVE\nChicago, Illinois 60608\n(41...",NaN,8:00 AM-3:35 PM,400057,41.866,-87.676,School Year 2016-2017,32,0,295,80,2,2,606,814,2,3,0,141,954,12,"50, 12",NaN,60608.0,"53,353"
4,No/unknown accessibility,Mrs.Sherita D Carter-King,NaN,N,16.7,NaN,Spanish,N,"6,7,8,9,10,11,12",21.4,NaN,NaN,"1321 S PAULINA ST\nChicago, Illinois 60608\n(4...",NaN,8:00 AM - 3:05 PM,609750,41.864,-87.669,School Year 2016-2017,0,0,28,2,0,0,8,37,0,0,0,6,38,2,NaN,NaN,60608.0,"53,353"
5,Generally accessible,Mr. Patrick Robinson,4:35AM - 6:30PM,N,16.6,N,Spanish,Y,"9,10,11,12",64.1,NaN,NaN,"1326 W 14TH PL\nChicago, Illinois 60608\n(41.8...",NaN,8:30 AM-4:30 PM,400102,41.863,-87.660,School Year 2016-2017,0,0,220,0,0,0,3,197,2,0,0,59,225,0,"12, 60",NaN,60608.0,"53,353"
6,Fully Accessible,Ms. Linnea Garrett,3:40 PM - 5:00 PM,N,15.6,Y,Spanish,Y,"9,10,11,12",69.5,NaN,NaN,"1301 W 14TH ST\nChicago, Illinois 60608\n(41.8...",NaN,8:50 AM-3:40 PM,400091,41.863,-87.659,School Year 2016-2017,2,0,189,3,0,0,45,222,1,0,0,61,245,8,"12, 60",NaN,60608.0,"53,353"
7,Generally accessible,Ms. Carrie Spitz,4:00PM-5:30PM,N,21.9,N,"Russian, Spanish",Y,"9,10,11,12",90.3,NaN,NaN,"4131 W CORTLAND ST\nChicago, Illinois 60639\n(...",NaN,M - R 8:05AM. - 3:50PM F 8:05- 1:30PM,400054,41.915,-87.730,School Year 2016-2017,1,0,30,120,0,0,945,936,2,5,0,106,992,9,"53, 72, 73",NaN,60639.0,"51,037"
8,Fully Accessible,Ms.Emily Ann Feltes,NaN,Y,15.9,Y,"French, Spanish",N,"9,10,11,12",74.3,NaN,NaN,"4338 W WABANSIA AVE\nChicago, Illinois 60639\n...",NaN,7:45 AM-3:00 PM,609691,41.912,-87.735,School Year 2016-2017,5,0,153,182,0,0,777,902,5,3,0,158,954,11,"65, 72",NaN,60639.0,"51,037"
9,Fully Accessible,Leticia Hernandez,3:25PM - 6:00 PM,Y,15.0,Y,Spanish,Y,"7,8,9,10,11,12",69.8,NaN,NaN,"4343 W WRIGHTWOOD AVE\nChicago, Illinois 60639...",NaN,8:05-3:25,609716,41.928,-87.736,School Year 2016-2017,0,0,31,146,1,1,402,415,1,2,0,88,445,7,"74, 76",Milwaukee District North (MD-N),60639.0,"51,037"


In [90]:
df_2017 = df[df['School_Year'] == 'School Year 2017-2018']
df_2017_income = pd.read_csv('data/meanHHincome_2017.csv', header=None)
df_2017_income.columns = ['Zip_y', 'Zip_Mean_Income']
df_2017 = pd.merge(df_2017, df_2017_income, on = 'Zip_y' )

In [91]:
df_new = df_2016.append(df_2017)

In [92]:
df_new.sort_values('School_ID')

,ADA_Accessible,Administrator,After_School_Hours,Attendance_Boundaries,Average_ACT_School,Bilingual_Services,Classroom_Languages,Dress_Code,Grades_Offered_All,Graduation_Rate_School,Hard_Of_Hearing,Is_GoCPS_Participant,Location,Network,School_Hours,School_ID,School_Latitude,School_Longitude,School_Year,Student_Count_Asian,Student_Count_Asian_Pacific_Islander,Student_Count_Black,Student_Count_English_Learners,Student_Count_Ethnicity_Not_Available,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Hispanic,Student_Count_Low_Income,Student_Count_Multi,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Special_Ed,Student_Count_Total,Student_Count_White,Transportation_Bus,Transportation_Metra,Zip_y,Zip_Mean_Income
60,Generally accessible,Cecilia Baader,NaN,N,15.2,Y,Spanish,Y,"9,10,11,12",68.8,NaN,NaN,"5410 S STATE ST\nChicago, Illinois 60609\n(41....",NaN,7:40 AM-3:22 PM,400010,41.796,-87.626,School Year 2016-2017,0,0,259,22,0,1,49,302,1,0,0,67,314,4,"15, 29, 55",NaN,60609.0,"46,096"
80,Fully Accessible,Brenda Stolle,3:15 PM -4:00 PM,N,14.9,Y,"Spanish, Spanish for Heritage Speakers",Y,"9,10,11,12",67.9,NaN,NaN,"3986 W BARRY AVE\nChicago, Illinois 60618\n(41...",NaN,8:15 AM - 3:15 PM,400013,41.937,-87.727,School Year 2016-2017,3,0,12,103,7,0,350,358,1,0,0,95,378,5,"53, 77, 56",NaN,60618.0,"90,510"
74,NaN,Brenda Stolle,3:15 PM -4:00 PM,N,NaN,Y,"Spanish, Spanish for Heritage Speakers",Y,"9,10,11,12",79.2,NaN,Y,"3986 W BARRY AVE\nChicago, Illinois 60618\n(41...",Charter,8:15 AM - 3:15 PM,400013,41.937,-87.727,School Year 2017-2018,2,0,7,108,0,0,323,316,2,1,0,86,340,5,"53, 77, 56",NaN,60618.0,"97,946"
101,Generally accessible,Mr. Michael Wang,NaN,N,19.4,N,"French, Spanish",N,"9,10,11,12",81.1,NaN,NaN,"2714 W AUGUSTA BLVD\nChicago, Illinois 60622\n...",NaN,8:00 AM-5:00 PM,400022,41.899,-87.695,School Year 2016-2017,15,0,238,5,27,2,202,332,17,1,0,38,621,119,"66, 70, 52, 49",Union Pacific North (UP-N),60622.0,"111,615"
95,NaN,Mr. Michael Wang,NaN,N,NaN,N,"French, Spanish",N,"9,10,11,12",84.6,NaN,Y,"2714 W AUGUSTA BLVD\nChicago, Illinois 60622\n...",Contract,8:00 AM-5:00 PM,400022,41.899,-87.695,School Year 2017-2018,11,0,207,5,38,1,216,193,16,2,0,40,612,121,"66, 70, 52, 49",Union Pacific North (UP-N),60622.0,"119,502"
44,Generally accessible,Ms. Kimberly Hinton,NaN,Y,16.3,N,Spanish,Y,"9,10,11,12",78.6,NaN,NaN,"1817 W 80TH ST\nChicago, Illinois 60620\n(41.7...",NaN,8:00 AM-3:30 PM,400032,41.748,-87.669,School Year 2016-2017,0,0,417,0,4,0,3,391,1,0,0,80,425,0,"9, 48, 79, 169",NaN,60620.0,"45,007"
44,NaN,Ms. Taquia Hylton,NaN,N,NaN,Y,Spanish,Y,"9,10,11,12",77.8,NaN,Y,"1817 W 80TH ST\nChicago, Illinois 60620\n(41.7...",Charter,8:00 AM-3:30 PM,400032,41.748,-87.669,School Year 2017-2018,0,0,347,0,0,0,1,298,0,0,0,76,348,0,"9, 48, 79, 169",NaN,60620.0,"47,131"
60,NaN,Ms. Angel Johnson,3:30 PM - 6:00 PM,N,NaN,Y,NaN,Y,"3,4,5,6,7,8,9,10,11,12",73.3,NaN,Y,"1309 W 95TH ST\nChicago, Illinois 60643\n(41.7...",Charter,8:00 AM-3:20 PM,400033,41.721,-87.656,School Year 2017-2018,0,0,1272,13,0,0,8,1050,0,0,0,239,1281,1,"9, 95W",Rock Island District (RI),60643.0,"83,368"
64,No/unknown accessibility,Ms. Kenyatta Starks,3:30 PM - 6:00 PM,N,15.4,N,NaN,Y,"3,4,5,6,7,8,9,10,11,12",82.6,NaN,NaN,"1309 W 95TH ST\nChicago, Illinois 60643\n(41.7...",NaN,8:00 AM-3:20 PM,400033,41.721,-87.656,School Year 2016-2017,0,0,1307,9,8,0,8,1221,4,0,0,230,1329,2,"9, 95W",Rock Island District (RI),60643.0,"81,467"
53,NaN,Torry A. Bennett,3:30pm,N,NaN,Y,"French, Spanish",Y,"9,10,11,12",84.4,NaN,Y,"3900 W PETERSON AVE\nChicago, Illinois 60659\n...",Charter,8:00am,400034,41.990,-87.727,School Year 2017-2018,165,0,61,134,0,3,514,675,19,3,0,209,918,153,"53, 84",NaN,60659.0,"68,047"


In [93]:
print(list(df_new))

['ADA_Accessible', 'Administrator', 'After_School_Hours', 'Attendance_Boundaries', 'Average_ACT_School', 'Bilingual_Services', 'Classroom_Languages', 'Dress_Code', 'Grades_Offered_All', 'Graduation_Rate_School', 'Hard_Of_Hearing', 'Is_GoCPS_Participant', 'Location', 'Network', 'School_Hours', 'School_ID', 'School_Latitude', 'School_Longitude', 'School_Year', 'Student_Count_Asian', 'Student_Count_Asian_Pacific_Islander', 'Student_Count_Black', 'Student_Count_English_Learners', 'Student_Count_Ethnicity_Not_Available', 'Student_Count_Hawaiian_Pacific_Islander', 'Student_Count_Hispanic', 'Student_Count_Low_Income', 'Student_Count_Multi', 'Student_Count_Native_American', 'Student_Count_Other_Ethnicity', 'Student_Count_Special_Ed', 'Student_Count_Total', 'Student_Count_White', 'Transportation_Bus', 'Transportation_Metra', 'Zip_y', 'Zip_Mean_Income']


In [94]:
df_new.drop(columns = ['Is_GoCPS_Participant','Location', 'School_Latitude', 'School_Longitude', 'Average_ACT_School'], inplace=True)

In [114]:
df_new.sort_values('School_ID').head(20)

,ADA_Accessible,Administrator,After_School_Hours,Attendance_Boundaries,Bilingual_Services,Classroom_Languages,Dress_Code,Grades_Offered_All,Graduation_Rate_School,Hard_Of_Hearing,Network,School_Hours,School_ID,School_Year,Student_Count_Asian,Student_Count_Asian_Pacific_Islander,Student_Count_Black,Student_Count_English_Learners,Student_Count_Ethnicity_Not_Available,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Hispanic,Student_Count_Low_Income,Student_Count_Multi,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Special_Ed,Student_Count_Total,Student_Count_White,Transportation_Bus,Transportation_Metra,Zip_y,Zip_Mean_Income
60,Generally accessible,Cecilia Baader,NaN,N,Y,Spanish,Y,"9,10,11,12",68.8,NaN,NaN,7:40 AM-3:22 PM,400010,School Year 2016-2017,0,0,259,22,0,1,49,302,1,0,0,67,314,4,"15, 29, 55",NaN,60609.0,"46,096"
80,Fully Accessible,Brenda Stolle,3:15 PM -4:00 PM,N,Y,"Spanish, Spanish for Heritage Speakers",Y,"9,10,11,12",67.9,NaN,NaN,8:15 AM - 3:15 PM,400013,School Year 2016-2017,3,0,12,103,7,0,350,358,1,0,0,95,378,5,"53, 77, 56",NaN,60618.0,"90,510"
74,NaN,Brenda Stolle,3:15 PM -4:00 PM,N,Y,"Spanish, Spanish for Heritage Speakers",Y,"9,10,11,12",79.2,NaN,Charter,8:15 AM - 3:15 PM,400013,School Year 2017-2018,2,0,7,108,0,0,323,316,2,1,0,86,340,5,"53, 77, 56",NaN,60618.0,"97,946"
101,Generally accessible,Mr. Michael Wang,NaN,N,N,"French, Spanish",N,"9,10,11,12",81.1,NaN,NaN,8:00 AM-5:00 PM,400022,School Year 2016-2017,15,0,238,5,27,2,202,332,17,1,0,38,621,119,"66, 70, 52, 49",Union Pacific North (UP-N),60622.0,"111,615"
95,NaN,Mr. Michael Wang,NaN,N,N,"French, Spanish",N,"9,10,11,12",84.6,NaN,Contract,8:00 AM-5:00 PM,400022,School Year 2017-2018,11,0,207,5,38,1,216,193,16,2,0,40,612,121,"66, 70, 52, 49",Union Pacific North (UP-N),60622.0,"119,502"
44,Generally accessible,Ms. Kimberly Hinton,NaN,Y,N,Spanish,Y,"9,10,11,12",78.6,NaN,NaN,8:00 AM-3:30 PM,400032,School Year 2016-2017,0,0,417,0,4,0,3,391,1,0,0,80,425,0,"9, 48, 79, 169",NaN,60620.0,"45,007"
44,NaN,Ms. Taquia Hylton,NaN,N,Y,Spanish,Y,"9,10,11,12",77.8,NaN,Charter,8:00 AM-3:30 PM,400032,School Year 2017-2018,0,0,347,0,0,0,1,298,0,0,0,76,348,0,"9, 48, 79, 169",NaN,60620.0,"47,131"
60,NaN,Ms. Angel Johnson,3:30 PM - 6:00 PM,N,Y,NaN,Y,"3,4,5,6,7,8,9,10,11,12",73.3,NaN,Charter,8:00 AM-3:20 PM,400033,School Year 2017-2018,0,0,1272,13,0,0,8,1050,0,0,0,239,1281,1,"9, 95W",Rock Island District (RI),60643.0,"83,368"
64,No/unknown accessibility,Ms. Kenyatta Starks,3:30 PM - 6:00 PM,N,N,NaN,Y,"3,4,5,6,7,8,9,10,11,12",82.6,NaN,NaN,8:00 AM-3:20 PM,400033,School Year 2016-2017,0,0,1307,9,8,0,8,1221,4,0,0,230,1329,2,"9, 95W",Rock Island District (RI),60643.0,"81,467"
53,NaN,Torry A. Bennett,3:30pm,N,Y,"French, Spanish",Y,"9,10,11,12",84.4,NaN,Charter,8:00am,400034,School Year 2017-2018,165,0,61,134,0,3,514,675,19,3,0,209,918,153,"53, 84",NaN,60659.0,"68,047"


In [105]:
df_new['School_Year'].value_counts()

School Year 2016-2017    121
School Year 2017-2018    118
Name: School_Year, dtype: int64

In [106]:
ada = df_new[df_new['School_Year'] == 'School Year 2016-2017']

In [108]:
ada.ADA_Accessible.value_counts()

Generally accessible        66
Fully Accessible            33
No/unknown accessibility    22
Name: ADA_Accessible, dtype: int64

In [111]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 0 to 117
Data columns (total 32 columns):
ADA_Accessible                             121 non-null object
Administrator                              239 non-null object
After_School_Hours                         143 non-null object
Attendance_Boundaries                      239 non-null object
Bilingual_Services                         228 non-null object
Classroom_Languages                        228 non-null object
Dress_Code                                 239 non-null object
Grades_Offered_All                         239 non-null object
Graduation_Rate_School                     239 non-null float64
Hard_Of_Hearing                            11 non-null object
Network                                    116 non-null object
School_Hours                               234 non-null object
School_ID                                  239 non-null int64
School_Year                                239 non-null object
Student_Count_